# 02: Knowledge-Grounded Agent Basics

This notebook introduces the `KnowledgeGroundedAgent` class, which wraps Gemini
with Google Search grounding into a full-featured QA agent.

## Learning Objectives

- Create and configure a `KnowledgeGroundedAgent`
- Understand the agent's system instructions
- Use the agent for single-turn Q&A
- Explore the DeepSearchQA evaluation dataset

In [ ]:
# Setup: Load environment and configure rich console
from aieng.agent_evals import (
    create_console,
    display_example,
    display_info,
    display_response,
    display_success,
)
from dotenv import load_dotenv


console = create_console()
load_dotenv(verbose=True)

## 1. Creating a Knowledge Agent

In [ ]:
from aieng.agent_evals.knowledge_agent import KnowledgeGroundedAgent
from rich.panel import Panel
from rich.table import Table


# Create the agent
agent = KnowledgeGroundedAgent()

# Display configuration
config_table = Table(
    title="🤖 Agent Configuration", show_header=True, header_style="bold cyan"
)
config_table.add_column("Setting", style="cyan")
config_table.add_column("Value", style="white")
config_table.add_row("Model", agent.model)
config_table.add_row("Planner Model", agent.config.default_planner_model)
config_table.add_row("Worker Model", agent.config.default_worker_model)

console.print(config_table)

## 2. Understanding System Instructions

The agent uses carefully crafted system instructions to guide its behavior.

In [ ]:
from aieng.agent_evals.knowledge_agent.agent import KNOWLEDGE_AGENT_INSTRUCTIONS
from rich.markdown import Markdown


console.print(
    Panel(
        Markdown(KNOWLEDGE_AGENT_INSTRUCTIONS),
        title="📜 System Instructions",
        border_style="blue",
        subtitle="[dim]Guides agent behavior[/dim]",
    )
)

## 3. Single-Turn Q&A

Let's use the agent to answer some questions.

In [ ]:
# Example 1: Current events
question = "What are the most significant AI developments in January 2026?"

console.print(
    Panel(
        f"[bold green]{question}[/bold green]",
        title="❓ Question",
        border_style="green",
    )
)

with console.status("[cyan]🔍 Agent is researching...[/cyan]", spinner="dots"):
    response = agent.answer(question)

display_response(response, console=console, title="AI Developments January 2026")

In [ ]:
# Example 2: Factual question
question = "What countries have successfully landed spacecraft on the Moon?"

console.print(
    Panel(
        f"[bold green]{question}[/bold green]",
        title="❓ Question",
        border_style="green",
    )
)

with console.status("[cyan]🔍 Agent is researching...[/cyan]", spinner="dots"):
    response = agent.answer(question)

display_response(response, console=console, title="Moon Landing Countries")

## 4. Using the AsyncClientManager

For applications that need to manage client lifecycle, use `AsyncClientManager`.

In [ ]:
from aieng.agent_evals.knowledge_agent import AsyncClientManager


# Create a manager (lazy initialization)
manager = AsyncClientManager()

display_info(f"Initialized: {manager.is_initialized()}", console=console)

# Access the agent (triggers initialization)
managed_agent = manager.agent
display_info(f"After access: {manager.is_initialized()}", console=console)

# Use the agent
with console.status("[cyan]Querying...[/cyan]", spinner="dots"):
    response = managed_agent.answer("What is the speed of light?")

display_response(response, console=console, title="Quick Answer", show_queries=False)

# Cleanup
manager.close()
display_success("Manager closed", console=console)

## 5. Exploring the DeepSearchQA Dataset

The DeepSearchQA dataset contains 896 research questions for evaluating knowledge agents.

In [ ]:
from aieng.agent_evals.knowledge_agent import DeepSearchQADataset


# Load the dataset
with console.status("[cyan]Loading DeepSearchQA dataset...[/cyan]", spinner="dots"):
    dataset = DeepSearchQADataset()

# Display dataset info
info_table = Table(
    title="📊 DeepSearchQA Dataset", show_header=True, header_style="bold cyan"
)
info_table.add_column("Metric", style="cyan")
info_table.add_column("Value", style="white")
info_table.add_row("Total Examples", str(len(dataset)))
info_table.add_row("Categories", str(len(dataset.get_categories())))

console.print(info_table)

In [ ]:
# Display categories
categories = dataset.get_categories()

cat_table = Table(
    title="📁 Problem Categories", show_header=True, header_style="bold green"
)
cat_table.add_column("Category", style="white")
cat_table.add_column("Count", style="cyan", justify="right")

for cat in sorted(categories):
    count = len(dataset.get_by_category(cat))
    cat_table.add_row(cat, str(count))

console.print(cat_table)

In [ ]:
# Show a sample example using the shared display utility
example = dataset[0]

display_example(
    example_id=example.example_id,
    problem=example.problem,
    category=example.problem_category,
    answer=example.answer,
    answer_type=example.answer_type,
    console=console,
)

In [ ]:
# Get random samples
samples = dataset.sample(n=3, random_state=42)

console.print("[bold]📚 Random Samples from Dataset[/bold]\n")

for ex in samples:
    display_example(
        example_id=ex.example_id,
        problem=ex.problem[:300] + "..." if len(ex.problem) > 300 else ex.problem,
        category=ex.problem_category,
        answer=ex.answer,
        console=console,
    )

## 6. Testing the Agent on DeepSearchQA

Let's test the agent on a sample question from the dataset.

In [ ]:
# Pick an example
test_example = samples[0]

console.print(
    Panel(
        f"[bold]Testing on Example {test_example.example_id}[/bold]\n\n"
        f"[cyan]Category:[/cyan] {test_example.problem_category}\n"
        f"[cyan]Expected Answer:[/cyan] {test_example.answer}",
        title="🧪 Test Setup",
        border_style="yellow",
    )
)

# Ask the agent
console.print(
    Panel(
        f"[bold green]{test_example.problem}[/bold green]",
        title="❓ Question",
        border_style="green",
    )
)

with console.status("[cyan]🔍 Agent is researching...[/cyan]", spinner="dots"):
    response = agent.answer(test_example.problem)

display_response(response, console=console, title="Agent Response")

# Compare
contains_answer = test_example.answer.lower() in response.text.lower()
if contains_answer:
    console.print("\n[green]✓ CONTAINS EXPECTED ANSWER[/green]")
else:
    console.print("\n[yellow]⚠ Answer may differ[/yellow]")
console.print(f"[dim]Expected: {test_example.answer}[/dim]")

## Summary

In this notebook, you learned:

1. How to create and configure a `KnowledgeGroundedAgent`
2. The system instructions that guide agent behavior
3. How to use the agent for single-turn Q&A
4. How to explore the DeepSearchQA evaluation dataset

**Next**: In the next notebook, we'll explore multi-turn conversations and run systematic evaluations.

In [ ]:
console.print(
    Panel(
        "[green]✓[/green] Notebook complete!\n\n"
        "[cyan]Next:[/cyan] Open [bold]03_multi_turn.ipynb[/bold] to learn about multi-turn conversations and evaluation.",
        title="🎉 Done",
        border_style="green",
    )
)